# 02: Extracción de Tabla de Consulta (Lookup) BCN

**Propósito:** Este *notebook* *scrapea* el índice completo de parlamentarios (históricos y en ejercicio) de la Biblioteca del Congreso Nacional (BCN).

**Dependencias:** Ninguna. Este *notebook* corre de forma independiente.

**Salidas (Artifacts):**
* `data/01_raw/bcn_url_master_lookup.csv`: Un archivo CSV único que mapea los nombres de los parlamentarios (tal como aparecen en la BCN) a sus URLs biográficas. Este archivo será el "diccionario" de consulta para el *notebook* `03_Extraction_Biographies`.

In [1]:
import pandas as pd
from pathlib import Path
import sys
import logging

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.extraction_utils import scrape_bcn_list
except ImportError:
    logging.error("Error: No se pudo importar 'scrape_bcn_list' desde 'src/extraction_utils.py'.")
    logging.error("Asegúrese de haber movido la función a ese archivo.")
    raise

In [2]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"

# Archivo de salida
OUTPUT_FILE = DATA_DIR_RAW / "bcn_url_master_lookup.csv"

# --- Constantes de Scraping BCN ---
BASE_URL = "https://www.bcn.cl/historiapolitica/resenas_parlamentarias/index.html"

# Parámetros para ex-parlamentarios (ej. 1990-2018 como en su notebook)
PARAMS_EX = {
    "categ": "por_periodo",
    "periodo": "1990-2018", # (Ajuste el rango si es necesario)
}

# Parámetros para parlamentarios en ejercicio
PARAMS_EJERCICIO = {
    "categ": "en_ejercicio",
    "filtros": "1",
}

HEADERS = {
    "User-Agent": "Mozilla/5.0",
}

# Asegurarse que el directorio de datos exista
DATA_DIR_RAW.mkdir(parents=True, exist_ok=True)

logging.info(f"Ruta Raíz: {ROOT}")
logging.info(f"Directorio de Datos Raw: {DATA_DIR_RAW}")
logging.info(f"Archivo de Salida: {OUTPUT_FILE}")

2025-10-25 00:02:37,137 - INFO - Ruta Raíz: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-
2025-10-25 00:02:37,142 - INFO - Directorio de Datos Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw
2025-10-25 00:02:37,143 - INFO - Archivo de Salida: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\bcn_url_master_lookup.csv


## 2. Ejecución del Scraper

Se llama a la función de scraping para ambas categorías (ex-parlamentarios y en ejercicio).

In [3]:
try:
    logging.info("Iniciando scraping de ex-parlamentarios...")
    # (Ajuste pagina_final si sabe cuántas páginas son)
    df_ex = scrape_bcn_list(BASE_URL, PARAMS_EX, pagina_inicial=1, pagina_final=30) 
    logging.info(f"Scraping de ex-parlamentarios finalizado. {len(df_ex)} registros.")

    logging.info("Iniciando scraping de parlamentarios en ejercicio...")
    df_activos = scrape_bcn_list(BASE_URL, PARAMS_EJERCICIO, pagina_inicial=1, pagina_final=10)
    logging.info(f"Scraping de parlamentarios en ejercicio finalizado. {len(df_activos)} registros.")

except Exception as e:
    logging.error(f"ERROR FATAL durante el scraping: {e}")
    raise

2025-10-25 00:02:37,152 - INFO - Iniciando scraping de ex-parlamentarios...
2025-10-25 00:02:38,080 - INFO - Scrapeada página 1 con 44 items.
2025-10-25 00:02:39,614 - INFO - Scrapeada página 2 con 44 items.
2025-10-25 00:02:41,007 - INFO - Scrapeada página 3 con 44 items.
2025-10-25 00:02:42,582 - INFO - Scrapeada página 4 con 44 items.
2025-10-25 00:02:44,099 - INFO - Scrapeada página 5 con 44 items.
2025-10-25 00:02:45,618 - INFO - Scrapeada página 6 con 44 items.
2025-10-25 00:02:47,099 - INFO - Scrapeada página 7 con 44 items.
2025-10-25 00:02:48,691 - INFO - Scrapeada página 8 con 44 items.
2025-10-25 00:02:50,062 - INFO - Scrapeada página 9 con 44 items.
2025-10-25 00:02:51,544 - INFO - Scrapeada página 10 con 44 items.
2025-10-25 00:02:52,926 - INFO - Scrapeada página 11 con 44 items.
2025-10-25 00:02:54,275 - INFO - Scrapeada página 12 con 44 items.
2025-10-25 00:02:55,628 - INFO - Scrapeada página 13 con 44 items.
2025-10-25 00:02:57,019 - INFO - Scrapeada página 14 con 44 it

## 3. Consolidación y Guardado

Se combinan ambos DataFrames, se eliminan duplicados (en caso de que un parlamentario aparezca en ambas listas) y se guarda el archivo maestro.

In [5]:
if df_ex.empty and df_activos.empty:
    logging.warning("No se obtuvieron datos. El archivo de salida no se creará.")
else:
    # 1. Combinar
    df_total = pd.concat([df_ex, df_activos], ignore_index=True)
    
    # 2. Limpiar duplicados
    # Mantenemos la primera aparición (probablemente de la lista 'ex' que es más grande)
    df_total_limpio = df_total.drop_duplicates(subset=["nombre_en_lista", "url_wiki"], keep="first")
    
    logging.info(f"Total registros (combinados): {len(df_total)}")
    logging.info(f"Total registros (únicos): {len(df_total_limpio)}")

    # 3. Guardar
    try:
        df_total_limpio.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
        logging.info(f"Guardado exitosamente: {OUTPUT_FILE}")
        display(df_total_limpio.head())
    except Exception as e:
        logging.error(f"ERROR al guardar el archivo: {e}")

logging.info("--- Extracción de BCN Lookup finalizada ---")

2025-10-25 00:03:28,194 - INFO - Total registros (combinados): 828
2025-10-25 00:03:28,195 - INFO - Total registros (únicos): 623
2025-10-25 00:03:28,205 - INFO - Guardado exitosamente: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\bcn_url_master_lookup.csv


,nombre_en_lista,url_wiki,pagina
0,Aburto Ochoa Marcos Aurelio,https://www.bcn.cl/historiapolitica/resenas_pa...,1
1,Accorsi Opazo Enrique,https://www.bcn.cl/historiapolitica/resenas_pa...,1
2,Acevedo Saez María Candelaria,https://www.bcn.cl/historiapolitica/resenas_pa...,1
3,Acuña Cisternas Mario Alberto,https://www.bcn.cl/historiapolitica/resenas_pa...,1
4,Aedo Jeldres Eric Mariano,https://www.bcn.cl/historiapolitica/resenas_pa...,1


2025-10-25 00:03:28,216 - INFO - --- Extracción de BCN Lookup finalizada ---
